In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import root, fixed_point
import matplotlib.pyplot as plt

In [ ]:
TFtot = 2000
kon = 0.0016399 
koff = 0.34393
kbasal = 0.02612
kmax = 13.588
Kd = 956.75
n = 4.203
kdegR = 0.042116
ktrans = 1.4514
kdegP = 0.007

p =  kon, TFtot, koff, kbasal, kmax, n, Kd, kdegR, ktrans, kdegP

Kd_scaled = Kd / TFtot
kmax_scaled = kmax * kdegR / (kbasal + kmax)
kbasal_scaled = kbasal * kdegR / (kbasal + kmax)

k_alpha = kon * 1000 / koff
k_beta = koff 

p_scaled = k_alpha, k_beta, kbasal_scaled, kmax_scaled, n, Kd_scaled, kdegR

print(Kd_scaled, kmax_scaled, kbasal_scaled)

In [ ]:
def khammash_ode_scaled(t,x,p_scaled,I):
    if callable(I):
        I = I(t)

    # Parameters
    k_alpha, k_beta, kbasal_scaled, kmax_scaled, n, Kd_scaled, kdegR = p_scaled

    # Variables
    TFon, mRNA = x
    TFon = np.max([0,TFon])

    # k_alpha = kon / koff
    # k_beta = koff

    dTFondt = (I * k_alpha * (1 - TFon) - TFon) * k_beta
    # dTFondt = I * kon * (1 - TFon) - koff * TFon
    dmRNAdt = kbasal_scaled + kmax_scaled * (TFon ** n) / ((Kd_scaled ** n) + (TFon ** n)) - kdegR * mRNA

    return [dTFondt, dmRNAdt]


def khammash_ode(t,x,p,I):
    if callable(I):
        I = I(t)

    # Parameters
    kon, TFtot, koff, kbasal, kmax, n, Kd, kdegR, ktrans, kdegP = p

    # Variables
    TFon, mRNA = x
    TFon = np.max([0,TFon])
    mRNA = mRNA * (kbasal + kmax) / kdegR

    dTFondt = I * kon * (TFtot - (TFon  * TFtot)) - koff * (TFon  * TFtot)
    dmRNAdt = kbasal + kmax * ((TFon  * TFtot) ** n) / ((Kd ** n) + ((TFon  * TFtot) ** n)) - kdegR * mRNA

    dTFondt = dTFondt / TFtot
    dmRNAdt = dmRNAdt * kdegR / (kbasal + kmax)

    return [dTFondt, dmRNAdt]

def fsolve_func(x,period,I,max_step):
    tspan = [0,period]
    sol = solve_ivp(khammash_ode, t_span=tspan, y0=x, args=(p_scaled,I),
                        rtol=1e-9, atol=1e-12, 
                        # t_eval=np.linspace(tspan[0], tspan[1], 1000000), 
                        method='BDF',
                        first_step = 0.0001,
                        max_step=max_step,
                        )
    # return (sol.y[:,-1] - x[:])
    return sol.y[:,-1]
    # return (sol.y[:,-1] - x) * [1/10,10]

def fsolve_func_scaled(x,period,I,max_step):
    tspan = [0,period]
    sol = solve_ivp(khammash_ode_scaled, t_span=tspan, y0=x, args=(p_scaled,I),
                        rtol=1e-9, atol=1e-12, 
                        # t_eval=np.linspace(tspan[0], tspan[1], 1000000), 
                        method='BDF',
                        first_step = 0.0001,
                        max_step=max_step,
                        )
    return (sol.y[:,-1] - x[:])
    # return sol.y[:,-1]
    # return (sol.y[:,-1] - x) * [1/10,10]

def I_fun_base(t, u, period, Imax):
    remainder = t - np.floor(t/period) * period
    I_out = np.zeros(np.array(t).size)
    I_out[remainder < u * period] = Imax
    return I_out

def steady_mRNA_solve(u, period, Imax, x_guess=None):

    I = lambda t: I_fun_base(t, u, period, Imax)

    if x_guess is None:
        x_guess = [u,u]

    if u > 0:
        max_step = u*period
    else:
        max_step = period / 10

    # try:
    #     sol = fixed_point(fsolve_func_scaled, x_guess, args=(period,I,max_step), xtol=1e-9, maxiter=5000)
    # except:
    #     print('first one failed')
    #     x_guess = [u,u]
    #     sol = fixed_point(fsolve_func_scaled, x_guess, args=(period,I,max_step/10), xtol=1e-9, maxiter=5000)
    # steady = sol


    sol = root(fsolve_func_scaled, x_guess, args=(period,I,max_step), tol=1e-9)
    if not sol.success:
        x_guess = [u,u]
        sol = root(fsolve_func_scaled, x_guess, args=(period,I,max_step), tol=1e-9)

    steady = np.maximum(sol.x,[0,0])
    tspan = [0,period]
    sol = solve_ivp(khammash_ode_scaled, t_span=tspan, y0=steady, args=(p_scaled,I),
                        rtol=1e-9, atol=1e-12, 
                        # t_eval=np.linspace(tspan[0], tspan[1], 1000000), 
                        method='BDF',
                        first_step = 0.0001,
                        max_step=max_step,
                        )
    steady = np.trapz(sol.y, x=sol.t) / period

    return steady#, sol.success, sol.message, sol.fun

def opto_growth(t,x,p,u,TF_fun):
        # print(p)
        # assert False    

    # Parameters
    EL222Pars, commonPars, ampPars, invertPars = p
    k_alpha, k_beta, kbasal_scaled, kmax_scaled, n, Kd_scaled, kdegR = EL222Pars
    kdegPPDC, Ks, Y, KI_PDC, nI_PDC = commonPars
    mu_max_amp, k_trans_Amp, kdeg_Amp, K_Amp, n_Amp, k_trans_PDC_amp = ampPars
    mu_max_invert, k_trans_Rep, kdeg_Rep, K_Rep, n_Rep, k_trans_PDC_invert = invertPars

    x_guess = x
    global setpoint

    tau_mRNA = 1 / kdegR

    # Variables
    mRNA_invert, mRNA_amp, Rep, Amp, Enz_invert, Enz_amp, S, X_invert, X_amp = x
    S = np.max(S,0)
    Enz_invert = np.max((Enz_invert,0))
    Enz_amp = np.max((Enz_amp,0))
    Rep = np.max((Rep,0))
    Amp = np.max((Amp,0))

    mu_x_invert = mu_max_invert * Enz_invert * S / (Ks + S) * ((KI_PDC ** nI_PDC) / (KI_PDC ** nI_PDC + Enz_invert ** nI_PDC + 1e-10))
    mu_x_amp = mu_max_amp * Enz_amp * S / (Ks + S) * ((KI_PDC ** nI_PDC) / (KI_PDC ** nI_PDC + Enz_amp ** nI_PDC + 1e-10))

    TF = TF_fun(u)
    mRNAsetpoint = kbasal_scaled + kmax_scaled * (TF ** n) / ((Kd_scaled ** n) + (TF ** n)) / kdegR

    dmRNA_invert_dt = (mRNAsetpoint - mRNA_invert) / tau_mRNA
    dmRNA_amp_dt = (mRNAsetpoint - mRNA_amp) / tau_mRNA

    
    dRepdt = k_trans_Rep * mRNA_invert - (kdeg_Rep + mu_x_invert) * Rep
    dEnz_invert_dt = k_trans_PDC_invert * (K_Rep ** n_Rep / (K_Rep ** n_Rep + Rep ** n_Rep)) - (kdegPPDC + mu_x_invert) * Enz_invert

    
    dAmpdt = k_trans_Amp * mRNA_amp - (kdeg_Amp + mu_x_amp) * Amp
    dEnz_amp_optodt = k_trans_PDC_amp * (Amp ** n_Amp / (K_Amp ** n_Amp + Amp ** n_Amp)) - (kdegPPDC + mu_x_amp) * Enz_amp

    dSdt = - (mu_x_invert / Y) * X_invert - (mu_x_amp / Y) * X_amp
    dX_invert_dt = mu_x_invert * X_invert
    dX_amp_dt = mu_x_amp * X_amp

    # names = ['mRNA_invert', 'mRNA_amp', 'Rep', 'Amp', 'Enz_invert', 'Enz_amp', 'S', 'X_invert', 'X_amp']
    # for var, name in zip([mRNA_invert, mRNA_amp, Rep, Amp, Enz_invert, Enz_amp, S, X_invert, X_amp], names):
    #     if var<0:
    #         print('BIOOPY', name)
    #         assert False



    return [dmRNA_invert_dt, dmRNA_amp_dt, dRepdt, dAmpdt, dEnz_invert_dt, dEnz_amp_optodt, dSdt, dX_invert_dt, dX_amp_dt]


In [ ]:
from tqdm import tqdm
from scipy.interpolate import interp1d

run = True
if run:
    u_arr = np.flip(np.concatenate((np.array([0]),10 ** np.linspace(-3,0,100))))

    # u_arr = np.flip(10 ** np.linspace(-1,0,100))
    Enz_arr = np.zeros(u_arr.size)
    mRNA_arr = np.zeros(u_arr.size)
    TF_arr = np.zeros(u_arr.size)

    setpoint = None
    for i, u in tqdm(enumerate(u_arr), total=u_arr.size):
        # print(u)
        period = 100 / 60
        Imax = 1
        out = steady_mRNA_solve(u, period, Imax, x_guess=setpoint)
        mRNA_arr[i] = out[1]
        TF_arr[i] = out[0]

mRNA_fun = interp1d(u_arr, mRNA_arr, kind='cubic')
TF_fun = interp1d(u_arr, TF_arr, kind='cubic')


In [ ]:
# dmRNAdt = kbasal_scaled + kmax_scaled * (TFon ** n) / ((Kd_scaled ** n) + (TFon ** n)) - kdegR * mRNA
mRNA_arr_myownss = kbasal_scaled + kmax_scaled * (TF_arr ** n) / ((Kd_scaled ** n) + (TF_arr ** n)) / kdegR

fig = plt.figure(figsize=(3.5,3.5))
ax = fig.add_subplot(111)
ax.scatter(mRNA_arr, mRNA_arr_myownss, label='mRNA steady state')
ax.plot([0,1],[0,1], color='black', linestyle='--')

In [ ]:
# assert False

# EL222 Parameters
TFtot = 2000
kon = 0.0016399 
koff = 0.34393
kbasal = 0.02612
kmax = 13.588
Kd = 956.75
n = 4.203
kdegR = 0.042116
ktrans = 1.4514
kdegP = 0.007

Kd_scaled = Kd / TFtot
kmax_scaled = kmax * kdegR / (kbasal + kmax)
kbasal_scaled = kbasal * kdegR / (kbasal + kmax)

k_alpha = kon * 1000 / koff
k_beta = koff 

EL222Pars = k_alpha, k_beta, kbasal_scaled, kmax_scaled, n, Kd_scaled, kdegR

# Common Parameters
# kdegR = 0.042116
# kdegP = 12 / np.log(2) * 60
kdegPPDC = 12 / np.log(2) * 60
Ks = 0.05
Y = 0.5
nI_PDC = 2
KI_PDC = 1
commonPars = kdegPPDC, Ks, Y, KI_PDC, nI_PDC

# Invert Parameters
kdeg_Rep = 2 / np.log(2) * 60
K_Rep = 0.01
n_Rep = 0.7
mu_max_invert = 1
k_trans_Rep = (mu_max_invert + kdeg_Rep) * 10
k_trans_PDC_invert = (mu_max_invert + kdegPPDC) * 1
invertPars = mu_max_invert, k_trans_Rep, kdeg_Rep, K_Rep, n_Rep, k_trans_PDC_invert

# Amp Parameters
kdeg_Amp = 2 / np.log(2) * 60
K_Amp = 0.4
mu_max_amp = 1.
n_Amp = 0.8
k_trans_Amp = mu_max_amp + kdeg_Amp
k_trans_PDC_amp = (mu_max_amp + kdegPPDC) * 5
ampPars = mu_max_amp, k_trans_Amp, kdeg_Amp, K_Amp, n_Amp, k_trans_PDC_amp

p = EL222Pars, commonPars, ampPars, invertPars

guesses = ([1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 0, 0], [1, 1, 1, 1])

print(EL222Pars)
print(commonPars)
print(invertPars)
print(ampPars)

if True:
    lastsol = [1, 1, 1, 1]
    steady_state = []
    for i, u in tqdm(enumerate(u_arr), total=u_arr.size):
        def rootfunc(x):
            S = 2
            X_invert = 0.01
            X_amp = 0.01
            TF = TF_fun(u)
            # print(u,TF)
            # assert False
            mRNAsetpoint = kbasal_scaled + kmax_scaled * (TF ** n) / ((Kd_scaled ** n) + (TF ** n)) / kdegR
            mRNA_invert = mRNAsetpoint
            mRNA_amp = mRNAsetpoint
            Rep, Amp, Enz_invert, Enz_amp = x
            x_in = mRNA_invert, mRNA_amp, Rep, Amp, Enz_invert, Enz_amp, S, X_invert, X_amp
            return opto_growth(0,x_in,p, u, TF_fun)[2:-3]
        output = root(rootfunc, lastsol, tol=1e-10, method='hybr')
        if output.success is False:
            for i in range(4):
                guess = guesses[i]
                output = root(rootfunc, guess, tol=1e-10, method='hybr')
                if output.success:
                    lastsol = output.x
                    break
            if output.success is False:
                for i in range(100):
                    guess = np.random.uniform(0,1,4)
                    output = root(rootfunc, guess, tol=1e-10, method='hybr')
                    if output.success:
                        lastsol = output.x
                        break
            steady_state.append(output.x)
        else:
            lastsol = output.x
            steady_state.append(output.x)
    steady_state = np.array(steady_state)

In [ ]:
Rep_ss, Amp_ss, Enz_invert_ss, Enz_amp_ss = steady_state.T
mu_amp = mu_max_amp * Enz_amp_ss * 2 / (Ks + 2) * ((KI_PDC ** nI_PDC) / (KI_PDC ** nI_PDC + Enz_amp_ss ** nI_PDC + 1e-10))
mu_invert = mu_max_invert * Enz_invert_ss * 2 / (Ks + 2) * ((KI_PDC ** nI_PDC) / (KI_PDC ** nI_PDC + Enz_invert_ss ** nI_PDC + 1e-10))

In [ ]:
amp_data = np.array([0.120837503, 0.123422835, 0.180854158, 0.19976335, 0.212471916, 0.262908006, 0.265834766, 0.308231875, 0.300134524, 0.307690564, 0.349198597, 0.235627624, 0.328256455, 0.256070133, 0.271335288, 0.18874746, 0.170414488, 0.133141302])
amp_lights = np.array([1, 1, 3, 3, 6, 6, 8, 8, 10, 10, 10, 20, 20, 40, 40, 70, 70, 100]) / 100

invert_data = np.array([0.259118174, 0.249223514, 0.251594487, 0.163109986, 0.116035613, 0.071939681, 0.064949075, 0.089238774, 0.08784264, 1E-10, 0.0414111, 0.040288699, 0.033536204])
invert_lights = np. array([0, 0, 1, 1.667, 1.667, 5, 5, 5, 5, 100, 100, 100, 100])/100

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(121)
ax.plot(u_arr, mu_amp, label='Amp', color='blue', linewidth=2)
ax.scatter(amp_lights, amp_data, color='blue', marker='o', s=50)
ax.set_xscale('symlog', linthresh=0.01)

ax = fig.add_subplot(122)
ax.plot(u_arr, mu_invert, label='Invert', color='red', linewidth=2)
ax.scatter(invert_lights, invert_data, color='red', marker='o', s=50)
ax.set_xscale('symlog', linthresh=0.01)

In [ ]:
# assert False

In [ ]:
def makessDoseRespCurve(p, u_input, TF_fun):
    kdegPPDC = 12 / np.log(2) * 60
    kdegR = 0.042116
    kdeg_Rep = 2 / np.log(2) * 60
    kdeg_Amp = 2 / np.log(2) * 60
    Y = 0.5
    Ks = 0.05
    k_alpha, k_beta, kbasal_scaled, kmax_scaled, n, Kd_scaled, KI_PDC, nI_PDC, mu_max_invert, k_trans_Rep, K_Rep, n_Rep, k_trans_PDC_invert, mu_max_amp, k_trans_Amp, K_Amp, n_Amp, k_trans_PDC_amp = p

    EL222Pars = k_alpha, k_beta, kbasal_scaled, kmax_scaled, n, Kd_scaled, kdegR
    commonPars = kdegPPDC, Ks, Y, KI_PDC, nI_PDC
    invertPars = mu_max_invert, k_trans_Rep, kdeg_Rep, K_Rep, n_Rep, k_trans_PDC_invert
    ampPars = mu_max_amp, k_trans_Amp, kdeg_Amp, K_Amp, n_Amp, k_trans_PDC_amp
    p_in = EL222Pars, commonPars, ampPars, invertPars

    lastsol = [1, 1, 1, 1]
    steady_state = []
    for i, u in enumerate(u_input):
        def rootfunc(x):
            S = 2
            X_invert = 0.01
            X_amp = 0.01
            TF = TF_fun(u)
            # print(u,TF)
            # assert False
            mRNAsetpoint = kbasal_scaled + kmax_scaled * (TF ** n) / ((Kd_scaled ** n) + (TF ** n)) / kdegR
            mRNA_invert = mRNAsetpoint
            mRNA_amp = mRNAsetpoint
            Rep, Amp, Enz_invert, Enz_amp = x
            x_in = mRNA_invert, mRNA_amp, Rep, Amp, Enz_invert, Enz_amp, S, X_invert, X_amp
            return opto_growth(0,x_in,p_in, u, TF_fun)[2:-3]
        output = root(rootfunc, lastsol, tol=1e-10, method='hybr')
        if output.success is False:
            for i in range(4):
                guess = guesses[i]
                output = root(rootfunc, guess, tol=1e-10, method='hybr')
                if output.success:
                    lastsol = output.x
                    break
            if output.success is False:
                for i in range(100):
                    guess = np.random.uniform(0,1,4)
                    output = root(rootfunc, guess, tol=1e-10, method='hybr')
                    if output.success:
                        lastsol = output.x
                        break
            steady_state.append(output.x)
        else:
            lastsol = output.x
            steady_state.append(output.x)
    steady_state = np.array(steady_state)
    return steady_state

# mRNA_invert, mRNA_amp, Rep, Amp, Enz_invert, Enz_amp, S, X_invert, X_amp = x

In [ ]:
# EL222 Parameters
TFtot = 2000
kon = 0.0016399 
koff = 0.34393
kdegR = 0.042116
kbasal = 0.02612
kmax = 13.588
Kd = 956.75
n = 4.203
ktrans = 1.4514
kdegP = 0.007

Kd_scaled = Kd / TFtot
kmax_scaled = kmax * kdegR / (kbasal + kmax)
kbasal_scaled = kbasal * kdegR / (kbasal + kmax)

k_alpha = kon * 1000 / koff
k_beta = koff 

EL222Pars = k_alpha, k_beta, kbasal_scaled, kmax_scaled, n, Kd_scaled, kdegR

# Common Parameters
kdegPPDC = 12 / np.log(2) * 60
Ks = 0.05
Y = 0.5
nI_PDC = 2
KI_PDC = 1
commonPars = kdegPPDC, Ks, Y, KI_PDC, nI_PDC

# Invert Parameters
kdeg_Rep = 2 / np.log(2) * 60
K_Rep = 0.01
n_Rep = 0.7
mu_max_invert = 1
k_trans_Rep = (mu_max_invert + kdeg_Rep) * 10 #/ kdeg_Rep
k_trans_PDC_invert = (mu_max_invert + kdegPPDC) * 1 #/ kdegPPDC
invertPars = mu_max_invert, k_trans_Rep, kdeg_Rep, K_Rep, n_Rep, k_trans_PDC_invert

# Amp Parameters
kdeg_Amp = 2 / np.log(2) * 60
K_Amp = 0.4
mu_max_amp = 1.
n_Amp = 0.8
k_trans_Amp = (mu_max_amp + kdeg_Amp) #/ kdeg_Amp
k_trans_PDC_amp = (mu_max_amp + kdegPPDC) * 5 #/ kdegPPDC
ampPars = mu_max_amp, k_trans_Amp, kdeg_Amp, K_Amp, n_Amp, k_trans_PDC_amp

orig_pars =  k_alpha, k_beta, kbasal_scaled, kmax_scaled, n, Kd_scaled, KI_PDC, nI_PDC, mu_max_invert, k_trans_Rep, K_Rep, n_Rep, k_trans_PDC_invert, mu_max_amp, k_trans_Amp, K_Amp, n_Amp, k_trans_PDC_amp

p_guess = k_alpha, k_beta, kbasal_scaled, kmax_scaled, n, Kd_scaled, KI_PDC, nI_PDC, mu_max_invert, k_trans_Rep, K_Rep, n_Rep, k_trans_PDC_invert, mu_max_amp, k_trans_Amp, K_Amp, n_Amp, k_trans_PDC_amp
bounds = (0, np.inf)



steady_state  = makessDoseRespCurve(p_guess, u_arr, TF_fun)

In [ ]:
k_trans_PDC_amp

In [ ]:
Rep_ss, Amp_ss, Enz_invert_ss, Enz_amp_ss = steady_state.T
mu_amp = mu_max_amp * Enz_amp_ss * 2 / (Ks + 2) * ((KI_PDC ** nI_PDC) / (KI_PDC ** nI_PDC + Enz_amp_ss ** nI_PDC + 1e-10))
mu_invert = mu_max_invert * Enz_invert_ss * 2 / (Ks + 2) * ((KI_PDC ** nI_PDC) / (KI_PDC ** nI_PDC + Enz_invert_ss ** nI_PDC + 1e-10))

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(121)
ax.plot(u_arr, mu_amp, label='Amp', color='blue', linewidth=2)
ax.scatter(amp_lights, amp_data, color='blue', marker='o', s=50)
ax.set_xscale('symlog', linthresh=0.01)

ax = fig.add_subplot(122)
ax.plot(u_arr, mu_invert, label='Invert', color='red', linewidth=2)
ax.scatter(invert_lights, invert_data, color='red', marker='o', s=50)
ax.set_xscale('symlog', linthresh=0.01)

In [ ]:
from scipy.interpolate import interp1d
from scipy.optimize import minimize

def fsolve_fun(x_in, u_vals, TF_fun, info): 
    x = 10 ** np.array(x_in)

    output_states = makessDoseRespCurve(x, u_vals, TF_fun)
    _, _, Enz_invert_ss, Enz_amp_ss = output_states.T
    _, _, _, _, _, _, KI_PDC, nI_PDC, mu_max_invert, _, _, _, _, mu_max_amp, _, _, _, _ = x
    mu_amp = mu_max_amp * Enz_amp_ss * 2 / (1 + 2) * ((KI_PDC ** nI_PDC) / (KI_PDC ** nI_PDC + Enz_amp_ss ** nI_PDC + 1e-10))
    mu_invert = mu_max_invert * Enz_invert_ss * 2 / (1 + 2) * ((KI_PDC ** nI_PDC) / (KI_PDC ** nI_PDC + Enz_invert_ss ** nI_PDC + 1e-10))

    amp_error = 0
    inv_error = 0
    for i, u in enumerate(u_vals):
        if any(amp_lights == u):
            amp_error += np.sqrt(np.mean((mu_amp[i] - amp_data[amp_lights == u]) ** 2))
        if any(invert_lights == u):
            inv_error += np.sqrt(np.mean((mu_invert[i] - invert_data[invert_lights == u]) ** 2))

    # error = np.sqrt(amp_error**2 + inv_error ** 2)
    error = amp_error + inv_error

    if error < info['best_error']:
        info['best_error'] = error
        info['best_pars'] = x_in
        # assert False

    if info['Nfeval']%100 == 0:
        print('Nfeval: ', info['Nfeval'], 'AmpError', amp_error, 'InvError', inv_error, 'CurrentError', error, 'BestError: ', info['best_error'])
    info['Nfeval'] += 1

    return error

In [ ]:
# EL222 Parameters
TFtot = 2000
kon = 0.0016399 
koff = 0.34393
kdegR = 0.042116
kbasal = 0.02612
kmax = 13.588
Kd = 956.75
n = 4.203
ktrans = 1.4514
kdegP = 0.007

Kd_scaled = Kd / TFtot
kmax_scaled = kmax * kdegR / (kbasal + kmax)
kbasal_scaled = kbasal * kdegR / (kbasal + kmax)

k_alpha = kon * 1000 / koff
k_beta = koff 

EL222Pars = k_alpha, k_beta, kbasal_scaled, kmax_scaled, n, Kd_scaled, kdegR

# Common Parameters
kdegPPDC = 12 / np.log(2) * 60
Ks = 0.05
Y = 0.5
nI_PDC = 2
KI_PDC = 1
commonPars = kdegPPDC, Ks, Y, KI_PDC, nI_PDC

# Invert Parameters
kdeg_Rep = 12 / np.log(2) * 60
K_Rep = 0.01
n_Rep = 0.7
mu_max_invert = 1
k_trans_Rep = (mu_max_invert + kdeg_Rep) * 10 #/ kdeg_Rep
k_trans_PDC_invert = (mu_max_invert + kdegPPDC) * 1 #/ kdegPPDC
invertPars = mu_max_invert, k_trans_Rep, kdeg_Rep, K_Rep, n_Rep, k_trans_PDC_invert

# Amp Parameters
kdeg_Amp = 12 / np.log(2) * 60
K_Amp = 0.4
mu_max_amp = 1.
n_Amp = 0.8
k_trans_Amp = (mu_max_amp + kdeg_Amp) #/ kdeg_Amp
k_trans_PDC_amp = (mu_max_amp + kdegPPDC) * 5 #/ kdegPPDC
ampPars = mu_max_amp, k_trans_Amp, kdeg_Amp, K_Amp, n_Amp, k_trans_PDC_amp

orig_pars =  k_alpha, k_beta, kbasal_scaled, kmax_scaled, n, Kd_scaled, KI_PDC, nI_PDC, mu_max_invert, k_trans_Rep, K_Rep, n_Rep, k_trans_PDC_invert, mu_max_amp, k_trans_Amp, K_Amp, n_Amp, k_trans_PDC_amp

u_vals = np.unique(np.concatenate((amp_lights, invert_lights)))
p_guess = np.array([k_alpha, k_beta, kbasal_scaled, kmax_scaled, n, Kd_scaled, KI_PDC, nI_PDC, mu_max_invert, k_trans_Rep, K_Rep, n_Rep, k_trans_PDC_invert, mu_max_amp, k_trans_Amp, K_Amp, n_Amp, k_trans_PDC_amp])
p_guess = np.log10(p_guess)
# bounds = ((0,10), # KI_PDC
#           (0.1,10), # nI_PDC
#           (0.1,10), # mu_max_invert
#           (0.1,10), # k_trans_Rep
#           (0.1,10), # K_Rep
#           (0.1,10), # n_Rep
#           (0.1,10), # k_trans_PDC_invert
#           (0.1,10), # mu_max_amp
#           (0.1,10), # k_trans_Amp
#           (0.1,10), # K_Amp
#           (0.1,10), # n_Amp
#           (0.1,10), # k_trans_PDC_amp
#         )
bounds = ((-1,2), # k_alpha
          (-1,2), # k_beta
          (-5,0), # kbasal_scaled
          (-3,0), # kmax_scaled
          (-1,1), # n
          (-1,2), # Kd_scaled
          (-1,1), # KI_PDC
          (0,2),  # nI_PDC
          (-1,1), # mu_max_invert
          (1,5), # k_trans_Rep ###
          (-3,1), # K_Rep   ###
          (-2,1), # n_Rep
          (2,5), # k_trans_PDC_invert
          (-1,1), # mu_max_amp
          (1,5), # k_trans_Amp
          (-2,1), # K_Amp
          (-2,1), # n_Amp
          (2,5), # k_trans_PDC_amp
        )

lower_bounds = [b[0] for b in bounds]
upper_bounds = [b[1] for b in bounds]

print(p_guess > lower_bounds)
print(p_guess < upper_bounds)
print(k_trans_PDC_amp)
print(k_trans_Amp)
print(k_trans_PDC_invert)
print(k_trans_Rep)

In [ ]:
print(len(bounds))
print(p_guess.shape)

In [ ]:
p_guess = np.array([ 1.66207274,  0.14589803, -4.01737962, -1.34688748,  0.25166187,  0.99998917,
 -0.04838099,  1.00142616,  0.11256127,  4.21336537, -2.194669,   -0.14171475,
  2.95617607, -0.10052536,  2.78203823, -0.16345467, -0.41337114,  3.83413109])

In [43]:
import scipy.optimize

p_guess = info['best_pars']
info = {'Nfeval':0, 'best_error':np.inf, 'best_pars':p_guess}

# sol = minimize(fsolve_fun, p_guess, args=(u_vals,TF_fun,info,), method='Powell', options={'maxiter': 1000, 'maxfev': 1000}, bounds=bounds)
sol = scipy.optimize.dual_annealing(fsolve_fun, bounds, args=(u_vals,TF_fun,info,), maxiter=10000, seed=0, x0=p_guess, no_local_search=False, minimizer_kwargs={'method':'Powell', 'bounds':bounds}, initial_temp=100)
# sol = scipy.optimize.differential_evolution(fsolve_fun, bounds, args=(u_vals,TF_fun,info,), maxiter=10000, seed=0, popsize=10, tol=0.01, mutation=(0.5, 1), recombination=0.7, updating='deferred', workers=-1, polish=True, init='latinhypercube', atol=0)
# sol = scipy.optimize.basinhopping(fsolve_fun, p_guess, minimizer_kwargs={'args':(u_vals,TF_fun,info,), 'method':'Powell', 'bounds':bounds}, niter=10000, T=0.5, stepsize=0.5, interval=50)
print(sol.x)
print(sol.success)
print(sol.message)

Nfeval:  0 AmpError 0.2419764251726411 InvError 0.16560513861576948 CurrentError 0.4075815637884106 BestError:  0.4075815637884106


/tmp/ipykernel_2522056/2460119681.py:10: RuntimeWarning: invalid value encountered in power
  mu_amp = mu_max_amp * Enz_amp_ss * 2 / (1 + 2) * ((KI_PDC ** nI_PDC) / (KI_PDC ** nI_PDC + Enz_amp_ss ** nI_PDC + 1e-10))
/tmp/ipykernel_2522056/2460119681.py:11: RuntimeWarning: invalid value encountered in power
  mu_invert = mu_max_invert * Enz_invert_ss * 2 / (1 + 2) * ((KI_PDC ** nI_PDC) / (KI_PDC ** nI_PDC + Enz_invert_ss ** nI_PDC + 1e-10))
/tmp/ipykernel_2522056/2602466646.py:138: RuntimeWarning: overflow encountered in double_scalars
  mu_x_amp = mu_max_amp * Enz_amp * S / (Ks + S) * ((KI_PDC ** nI_PDC) / (KI_PDC ** nI_PDC + Enz_amp ** nI_PDC + 1e-10))


Nfeval:  100 AmpError 0.602667465459838 InvError 0.20397555012938104 CurrentError 0.806643015589219 BestError:  0.4059842733419674
Nfeval:  200 AmpError 0.23975209996925514 InvError 0.1660111893901497 CurrentError 0.40576328935940487 BestError:  0.40576328935940487
Nfeval:  300 AmpError 2.753027113157754 InvError 0.42425536395735314 CurrentError 3.1772824771151074 BestError:  0.4042324338768297
Nfeval:  400 AmpError 0.579907163727476 InvError 0.4977065475577593 CurrentError 1.0776137112852353 BestError:  0.4042324338768297


In [ ]:
print(info)
# print(sol.x)

In [ ]:
print(fsolve_fun(np.log10(orig_pars), u_vals, TF_fun, {'Nfeval':0, 'best_error':0, 'best_pars':p_guess}), orig_pars)
# print(fsolve_fun(sol.x, u_vals, mRNA_fun, {'Nfeval':0, 'best_error':np.inf, 'best_pars':p_guess}), sol.x)
print('======================================')
print(fsolve_fun(info['best_pars'], u_vals, TF_fun, {'Nfeval':0, 'best_error':0, 'best_pars':p_guess}), info['best_pars'])

In [ ]:
parstouse = 10 ** info['best_pars']
utouse = u_arr

output_states = makessDoseRespCurve(parstouse, utouse, TF_fun)
_, _, Enz_invert_ss, Enz_amp_ss = output_states.T
_, _, _, _, _, _, KI_PDC, nI_PDC, mu_max_invert, _, _, _, _, mu_max_amp, _, _, _, _ = parstouse
mu_amp = mu_max_amp * Enz_amp_ss * 2 / (1 + 2) * ((KI_PDC ** nI_PDC) / (KI_PDC ** nI_PDC + Enz_amp_ss ** nI_PDC + 1e-10))
mu_invert = mu_max_invert * Enz_invert_ss * 2 / (1 + 2) * ((KI_PDC ** nI_PDC) / (KI_PDC ** nI_PDC + Enz_invert_ss ** nI_PDC + 1e-10))

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(121)
ax.plot(utouse, mu_amp, label='Amp', color='blue', linewidth=2)
ax.scatter(amp_lights, amp_data, color='blue', marker='o', s=50)
ax.set_xscale('symlog', linthresh=0.01)

ax = fig.add_subplot(122)
ax.plot(utouse, mu_invert, label='Invert', color='red', linewidth=2)
ax.scatter(invert_lights, invert_data, color='red', marker='o', s=50)
ax.set_xscale('symlog', linthresh=0.01)

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
ax.plot(utouse, mu_invert, label='Invert', color='red', linewidth=2)
ax.plot(utouse, mu_amp, label='Amp', color='blue', linewidth=2)
ax.set_xscale('symlog', linthresh=0.01)


In [ ]:
parstouse